# Notebook 15: Correlation and Missing Data
by Andrea Lommen and Rachel Langgin \\
March 2022 and July 2024 \\
Haverford College and University of Nevada, Las Vegas \\

In this notebook we demonstrate how you might quantify the statistical significance of a correlation in your data. You may wish to use this tool in your project. I've given you enough here to start working with it.

**Calculating Correlations**
For your projects you may want to explore finding correlations between sets of data.  This is a good reference for this idea:

https://blogs.oracle.com/datascience/introduction-to-correlation

It’s a bit too much information. Read the Introduction. Skip the “Types of Correlation.” We will use the Pearson Correlation coefficient (that’s the second method they show), so only read that one. Within that section just work to understand the ‘original’ formulation (the first formula.) And then skip down to the section “Calculating Correlation in Pandas”. Also read “Correlation and Causation.”  
    
This notebook shows an example of calculating a correlating coefficient and using it to make statements about the correlation of two data sets.

I'll use the SDSS query we used before, but I just want to test for a correlation between the u, g, r, and i colors of the galaxies, so I'll only get those columns.

In [ ]:
# This cell is all stuff you've seen before
# Import Python libraries to work with SciServer
import SciServer.CasJobs as CasJobs # query with CasJobs
import SciServer.SciDrive as SciDrive   # read/write to/from SciDrive
import SciServer.SkyServer as SkyServer   # show individual objects and generate thumbnail images through SkyServer
print('SciServer libraries imported')

# Import other libraries for use in this notebook.
import numpy as np                  # standard Python lib for math ops
# from scipy.misc import imsave       # save images as files
import pandas as pd                     # data manipulation package
import matplotlib.pyplot as plt     # another graphing package
import os                           # manage local files in your Compute containers
print('Supporting libraries imported')

import astroML
from astroML.datasets import fetch_sdss_spectrum
from astropy.io import ascii

# Apply some special settings to the imported libraries
# ensure columns get written completely in notebook
pd.set_option('display.max_colwidth', -1)
# do *not* show python warnings
import warnings
warnings.filterwarnings('ignore')
print('Settings applied')

query="""
SELECT p.objId,
    p.dered_u as u, p.dered_g as g, p.dered_r as r, p.dered_i as i,
    s.z, p.ra, p.petror90_r
FROM galaxy AS p
    JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE p.petror90_r > 10
  and p.ra between 100 and 250
  and s.z between 0.02 and 0.5
  and p.g < 17
"""
#Then, query the database. The answer is a table that is
# being returned to a dataframe that we've named all_gals.
all_gals = CasJobs.executeQuery(query, "dr14")


They are read in as pandas data frames, which is convenient, because pandas has a built-in "corr" method that will calculate the correlation for us.  Before we do that, though, let's plot r (red) vs g (green) to see if we expect any correlation between the two.

In [ ]:
plt.plot(all_gals.u, all_gals.g, '.')
plt.xlabel('magnitude in a red filter')
plt.ylabel('magnitude in a blue filter')

So we would call that a strong correlation!! As the red magnitude increases, so does the blue magnitude!

In [ ]:
# Now you have some options about how to use the "corr" method
# in Pandas.
# To calculate the Pearson correlation coefficient between
# two columns in the same data frame do this:

all_gals['g'].corr(all_gals['u'], method='pearson')

# Note that I think you can pass any array you want as the
# argument to corr, and it doesn't have to be a dataframe, but
# I haven't actually checked that out yet.

# Note that it follows the pattern for using a method.  The "object" is all_gals.
# You may not have realized that "all_gals" was an object you created.  It's an object
# in the pandas dataframe class. So to use a method, you say object.method(argument).

In [ ]:
# Note that by definition, the correlation of one column with another is the same as
# the correlation of the second with the first.  So all I did here was switch the
# order.
all_gals['u'].corr(all_gals['g'], method='pearson')


That number is pretty close to 1, so we say those data are highly correlated.

In [ ]:
# Alternatively, you can make a table of all the correlations
# between all the pairs of columns in your data frame!
# I think this is the coolest. It demonstrated the power of a dataframe, a method, and
# a computer in general.
all_gals.corr()

In [ ]:
# column i with column petror90_r is a negative correlation.  Let's
# see what that looks like.
plt.plot(all_gals.i, all_gals.petror90_r, '.')

So notice that u has a correlation of "1" with itself (as does g,r,i, etc).  So the closer the correlation coefficient is to 1 the more the columns are correlated. ObjId really has nothing to do physically with the object. It's just a name, so you expect 0 correlation with anything.  You don't get 0, but you get a bunch of small numbers (-.02, etc) for anything correlated with ObjId.

Pandas actually lets you color code this table so you can easily see where the large correlations are. Check this out:

In [ ]:
all_gals.corr(method = 'pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

So the dark red squares represent columns that are highly correlated with each other, and the dark blues....hmmm it doesn't actually look completely consistent to me.  The very light colors represent not very correlated pairs - those are mostly close to 0.

So that's how you use the "corr" method in pandas.  

# Now let's back up and talk about what a correlation actually is

One nice podcast I recommend is an by EPHS called "Correlation and Correlation Coefficient, Lecture 5 and "Establishing Causation" Lecture 20 on this page:
https://itunes.apple.com/za/podcast/ephs-statistics-chapters-3-4/id695094178?mt=2  They're free if you'd like to watch them.

Correlation describes the relationship between two variables that have a linear relationship or that you suspect might have a linear relationship. For example, you may suspect that chocolate consumption per capita is directly related to Nobel Prizes per capita.

https://www.businessinsider.com/chocolate-consumption-vs-nobel-prizes-2014-4
Here's another article about it.  I think the second article is somewhat more responsible and also a great read, but it doesn't show the plot.
https://www.reuters.com/article/us-eat-chocolate-win-the-nobel-prize/eat-chocolate-win-the-nobel-prize-idUSBRE8991MS20121010

The correlation coefficient attempts to quantify that - to put a number on the strength of that correlation.  The strength of the correlation on the graph above is r=.79.  Pretty strong!  (Correlation doesn't necessarily mean causation.)

One quality of correlation coefficients is that it doesn't matter which thing you plot vs which - which you put on the x-axis and which you put on the y-axis - you'll get the same correlation coefficient.

Here's the formula:

$$
r = \frac{\sum(x_i - avg(x)(y_i-avg(y))}{\sqrt{(\sum(x_i-avg(x))^2\sum(y_i-avg(y))^2}}
$$

I will not test you on this formula!  But I knew some of you would want to see it. I thought the concept would be useful to many of you as you complete your projects.  You can kinda see how the formula works.  Being able to
think like a programmer helps.  When i=0, so your first data point, if x is greater than average and y is greater than average, then that data point will contribute positively to the correlation.

It's defined such that when r is 1 it's highly correlated, and when r =0 it's not.

A linear relationship is something like what we plotted in the last lab: $y = mx + b$, where $x$ is one of your variables and $y$ is the other variable.  The other way to say it that as one goes up the other goes up.  Or as one goes up the other goes down.


So now, guess the strength of the correlation you see here.  How strong is the evidence that Breaking Bad really does get more people watching over the course of each season?
https://www.wired.com/2014/03/graph-quality-every-tv-show-ever-amazing-site/


Now play this game with your partner.
http://guessthecorrelation.com/


Correlation does not imply causation. (The divorce rate in Maine is correlated with Margarine consumption)
https://www.bbc.com/news/magazine-27537142

It has to be linear. Some of you have relationships that aren't linear in time (it wavers) but it may be that one of your variables mimics another.  And if you plotted those two versus each other the correlation would be large.

### How do you calculate correlation if you're not using a pandas data frame?

I would use
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html

Let me know if you need help.  You'll have to import scipy.stats and then you just call pearsonr(x,y) where
x and y are the data sets you want to correlate.

The first number it returns is the pearson r.

Let's try this on some fake data.  Two sets of random numbers.

In [ ]:
import scipy.stats as st
x = np.random.normal(0,1,100)
y = np.random.normal(0,1,100)
st.pearsonr(x,y)

In [ ]:
### Now let's try correlating data with itself.  It should give high correlation!
st.pearsonr(y,y)

### What if I have missing data?

What's the problem with missing data?

Is it missing randomly?  Or is it missing uniformly?  

Let me give you an example of a faulty conclusion caused by missing data.  Let's say that you're calculating
the correlation between education and joblessness, but you're missing all the data from people who had less
education. That happened because you sent out your questionnaire using college's alumni databases, so the people with less education didn't participate in your survey.  You may conclude that there's no correlation and you should quit school right now.



### Missing data indicators:
There are different common ways of indicating missing data, and they work in different scenarios:
* empty string: “”
* question mark or other indicator string:
* “?”
* “MISSING”
* “UNKNOWN”

Choose a specific value that indicates missing and is of the required type.  Common choices include:
* -1
* 0
* 999

### Missing vs not applicable
Missing data items are different than items that were not applicable.

Missing: this could be data collection error at random or a systematic lack of collection – it’s missing and therefore unknown.

Not applicable: this is relevant information to the data – the attribute is known to be not applicable to the item.

N/A data can be handled by creating a "different" specific value from above to use to filter it out.

### Missing data strategies
What do we do when an algorithm needs to handle a missing value?

Have the algorithm check for a missing item indicator and handle it specially.  If different missing item indicators are used per attribute, then each attribute needs a special check.

Fill in the missing items somehow.  Keep track of which items were filled in (as a new attribute), or keep a separate copy of the data. Never modify the original data!


#### One option:  Ignore / remove the data
If:
* the data is missing uniformly at random and
* not too much data is missing
* it might not be harmful to ignore or remove the missing items.

Possibilities:
* Remove rows with lots of missing items.
* Remove columns with lots of missing items.


#### Another option: modification, Replace with the mean of that "attribute"
For each missing item:
* find the mean or mode of the attribute
* replace the missing item with that mean or mode

Pros:
* the item is the right type
* the true value is likely not too far from the replaced value

Cons:
* the data looks more like the mean / mode than it should
* requires a separate analysis of why / which data is missing


#### Another option: modification, Replace with a random item
For each missing item:
* find all the possible values the attribute takes in the data
* pick one of the possible values uniformly at random and replace the missing item with it

Pros:
* the item is the right type
* the data as a whole isn’t pulled towards the mean / mode

Cons:
* the true value could be very far from the value it’s replaced with
* requires a separate analysis of why / which data is missing


### In all cases:
Test to make sure that your conclusion is not dependent upon what you chose to do with your missing data.
* Make a different choice above and see what the conclusion is

## Assignment

* Download a csv off the web OR **use one of the ones from one of your projects**.  I recommend the second option, because it'll let you talk to your partner about your project a little bit.
* Create a class that contains methods that do the following:
    (a) reads in a csv (the argument should be the filename)
    (b) plots two columns (the arguments should be the two column names)and prints their correlation coefficient.
    (c) finds the correlation between all the columns in the csv (this method should have no arguments)
* If you've created a sufficiently flexible class, you should be able to use it on a different csv file.  Try the other CSV file (the one from the partner whose CSV file you weren't using.)
* If there's data missing in your table, employ one of the techniques I talked about on Monday to deal with it.  For example, replace the missing data with the mean of the rest of the data.  Create a method that does this.

# Merging Data here's one thing to try:

https://www.statology.org/pandas-merge-on-index/